In [1]:
from pyspark.sql import SparkSession
from google.cloud import storage

In [2]:
spark = SparkSession.builder.appName('Ops').config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7de759eb-ab08-4d37-bc3e-dd33188e2f14;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.1 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 727ms :: artifacts dl 27ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.1 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-runtime;3.5.2 fro

In [3]:
from delta import *

In [4]:
bucket = "data-lake-jlr-gold-zone"
prefix = "dev/data"

In [5]:
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_or_name=bucket, prefix='dev/data/', delimiter='_delta_log/')

In [6]:
tables = []
for blob in blobs:
    if blob.name.endswith('/'):
        table = blob.name.rsplit('/', 3)[1]
        print(table)
        tables.append(table)

data
documentedname
endcustomerlocation
feature
featuregroup
legalentity
location
model
news
order
product_l0
product_l1
product_l2
sanctionmeasure
sanctionparty
sanctions
sob
supplier


In [7]:
tables = tables[1:]
tables

['documentedname',
 'endcustomerlocation',
 'feature',
 'featuregroup',
 'legalentity',
 'location',
 'model',
 'news',
 'order',
 'product_l0',
 'product_l1',
 'product_l2',
 'sanctionmeasure',
 'sanctionparty',
 'sanctions',
 'sob',
 'supplier']

In [ ]:
for table in tables:
    deltaTable = DeltaTable.forPath(spark, f"gs://{bucket}/{prefix}/{table}")
    deltaTable.generate("symlink_format_manifest")

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/02/22 19:10:35 WARN hive.metastore: Failed to connect to the MetaStore Server...


In [ ]:
spark.sql("""CREATE DATABASE IF NOT EXISTS supply_chain""")

In [ ]:
spark.sql("""SHOW DATABASES""").show()

In [ ]:
for table in tables:
    df = spark.read.format('delta').load(f'gs://{bucket}/{prefix}/{table}')
    
    data_types = ''
    for col in df.dtypes:
        data_types = data_types + col[0] + ' ' + col[1] + ', '
    
    data_types = data_types[:-2]
    print(f'Nombre_table: {table}, tipos_de_datos: {data_types}\n')
    
    spark.sql(f"""CREATE TABLE supply_chain.{table} ({data_types}) 
              ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
              STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
              OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
              LOCATION 'gs://{bucket}/{prefix}/{table}/_symlink_format_manifest'
              TBLPROPERTIES(delta.compatibility.symlinkFormatManifest.enabled=true)""")